In [27]:
import cv2
import urllib
import numpy as np
#from urllib.request import urlopen
#import random
from random import randrange, uniform

#check version
print("OpenCv version: ", cv2.__version__)

OpenCv version:  4.5.3


In [28]:
#bi_thd = 30
#bi_max = 255

# Parameters for Hough line detection
rho = 1
theta = np.pi/180
threshold = 80
min_line_length = 10
max_line_gap = 100

#img_path = "D:/Research/opencv/image/dora/b.jpg"
img_path = "D:/Research/opencv/image/dora/stitch_test.jpg"

In [29]:
def displayResizedImg(name, img, factor):
    imgDisplay = cv2.resize(img, None, fx=factor, fy=factor, interpolation=cv2.INTER_AREA)
    cv2.imshow(name, imgDisplay)

In [30]:
#read from file
img = cv2.imread(img_path)

In [31]:
#Convert image to gray
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#ret, binary = cv2.threshold(gray, bi_thd, bi_max, cv2.THRESH_BINARY)
#binary = cv2.inRange(gray, 90, 120)

# Apply an adaptive threshold
# ADAPTIVE_THRESH_MEAN_C
# ADAPTIVE_THRESH_GAUSSIAN_C
# 
#binary = cv2.adaptiveThreshold(gray,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
binary = cv2.inRange(gray, 90, 180)

In [32]:
#find and draw contours
#contours, hierarchy = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
#img_con = cv2.drawContours(img.copy(),contours,-1,(0,255,255),1)

In [33]:
#for i in contours[0]:
#    color = np.array( [uniform(0, 256), uniform(0,256), uniform(0,256)], dtype = "uint16")
#    img_con = cv2.drawContours( img.copy(), contours, int(i), color, 2, cv2.LINE_8, hierarchy, 0 )

In [34]:
#Crop image
showCrosshair = False
fromCenter = False
# Select ROI
rect = cv2.selectROI("select roi", gray, showCrosshair, fromCenter)
x, y, w, h = rect

# Crop image
imCropBin = gray[y : y+h, x:x+w]
imCrop = img[y : y+h, x:x+w]

In [35]:
#Detect line
fld = cv2.ximgproc.createFastLineDetector(10, 1.414, 90.0, 180.0, 3, False)
lines = fld.detect(imCropBin)
drawn_img = fld.drawSegments(imCropBin,lines)

In [36]:
#Detect line Canny
imCanny = cv2.blur(imCropBin, (3,3))
edges = cv2.Canny(imCanny, 90.0, 180.0)

In [ ]:
# detect Gradient
imCropBin = cv2.blur(imCropBin, (3,3))
#sobelxy = cv2.Sobel(imCropBin, cv2.CV_32F, 1, 1, 3)
#kernel = np.ones((3,3), np.uint8)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
#open_result = cv2.morphologyEx(sobelxy, cv2.MORPH_OPEN, kernel, iterations=1)
gradient = cv2.morphologyEx(imCropBin, cv2.MORPH_GRADIENT, kernel, iterations=2)
#sobelxy = cv2.dilate(sobelxy, kernel, iterations = 1)
#sobelxy = cv2.dilate(sobelxy, kernel, iterations = 1)
#sobelxy = cv2.erode(sobelxy, kernel, iterations = 1)
#sobelxy = cv2.erode(sobelxy, kernel, iterations = 1)
#sobelxy = cv2.resize(sobelxy, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
#cv2.imshow('Sobel Edge', sobelxy)
#cv2.imshow('Open', open_result)
#gradient = cv2.adaptiveThreshold(gradient,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,2)
#displayResizedImg('Gradient', gradient, 5)

gradient = cv2.resize(gradient, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
#cv2.imshow('Gradient', gradient)

gradient = cv2.morphologyEx(gradient, cv2.MORPH_OPEN, kernel, iterations=5)
gradient = cv2.erode(gradient, kernel, iterations = 4)
ret, gradient = cv2.threshold(gradient, bi_thd, bi_max, cv2.THRESH_BINARY)
gradient = cv2.erode(gradient, kernel, iterations = 7)

edges = cv2.Canny(gradient, bi_thd, 255.0)


lines = cv2.HoughLinesP(edges,rho,theta,threshold,np.array([]),min_line_length, max_line_gap)
imCropA = cv2.resize(imCrop, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)

y_list = []
font = cv2.FONT_HERSHEY_SIMPLEX
for line in lines:
        x1,y1,x2,y2 = line.reshape(4)
        color = int(y1)
        cv2.line(imCropA, (x1,y1), (x2,y2), (0,0,color), 3)
        x = int((x1+x2)/2)
        cv2.putText(imCropA,'{}'.format(y1), (x, y1+10), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
        y_list.append(y1)

#Group data
y_list = sorted(y_list)
print(y_list)


#gradient = cv2.dilate(gradient, kernel, iterations = 1)
#gradient = cv2.resize(gradient, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)

#cv2.imshow('Processed', gradient)
cv2.imshow('imCrop', imCropA)
cv2.imshow('edge', edges)

[40, 45, 48, 53, 105, 113, 118, 165, 170, 173, 178, 230, 235, 235, 238, 243]


In [ ]:
#show image 
#drawn_img = cv2.resize(drawn_img, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
#edges = cv2.resize(edges, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)

cv2.imshow('Original image', img)
#cv2.imshow('Selected ROI', drawn_img)
displayResizedImg('ROI', imCropBin, 5)
#imCropBin = cv2.resize(imCropBin, None, fx=5, fy=5, interpolation=cv2.INTER_AREA)
#cv2.imshow('ROI', imCropBin)

#cv2.imshow('Canny Edge', edges)
#cv2.imshow('roi', imCrop)

In [ ]:
#wait keydown and then close all windows
cv2.waitKey()
cv2.destroyAllWindows()